# LAB 3 - UART integration

In [27]:
import serial.tools.list_ports
ports = list(serial.tools.list_ports.comports())
for p in ports:
    print(p)

/dev/ttyUSB0 - USB Serial
/dev/ttyUSB1 - TUL - TUL


In [34]:
import serial
ser = serial.Serial('/dev/ttyUSB0', 115200, timeout=10)

First, create two random operands (A,B) that have OPERAND_WIDTH length.

In [1]:
import random
import time

OPERAND_WIDTH = 512

N_BYTES = int(OPERAND_WIDTH/8)

A = random.randrange(2**(OPERAND_WIDTH-1), 2**OPERAND_WIDTH-1)
B = random.randrange(2**(OPERAND_WIDTH-1), 2**OPERAND_WIDTH-1)

if A < B:
    A, B = B, A

res_sum = A + B
res_sub = A - B

print("A     = ", hex(A))
print("B     = ", hex(B))
print("A + B = ", hex(res_sum))
print("A - B = ", hex(res_sub))

A_bytes = bytearray.fromhex(format(A, 'x'))
B_bytes = bytearray.fromhex(format(B, 'x'))

A     =  0xe220339f70580b619d81d58c3c1a37ed26207438266dbc2cc588268513a0879f11a04499b8755c6b587ad20fe71bc2cd9d5270c50d52d2729d912ddceb876fec
B     =  0xb2dd28b4f446ae30ce3e7a5b9c72c110d6fe7b1b8e40ba55b716dcb33747951051914d04b5883560e2c47205fc3fc8e15de1bfe0b5ca31198a511a03dcab5cbd
A + B =  0x194fd5c54649eb9926bc04fe7d88cf8fdfd1eef53b4ae76827c9f03384ae81caf6331919e6dfd91cc3b3f4415e35b8baefb3430a5c31d038c27e247e0c832cca9
A - B =  0x2f430aea7c115d30cf435b309fa776dc4f21f91c982d01d70e7149d1dc58f28ec00ef79502ed270a75b66009eadbf9ec3f70b0e45788a159134013d90edc132f


Then, send the two operands (A,B) via the serial port, byte by byte. Later, the PYNQ board receives the result of the addition, and transmits it back to the computer.

In [38]:
for op_byte in A_bytes:
    hex_byte = ("{0:02x}".format(op_byte))
    ser.write(bytearray.fromhex(hex_byte))
    time.sleep(0.001)

for op_byte in B_bytes:
    hex_byte = ("{0:02x}".format(op_byte))
    ser.write(bytearray.fromhex(hex_byte))
    time.sleep(0.001)

res_rcvd = ser.read(N_BYTES+1) 

res2 = int.from_bytes(res_rcvd, "big")

if res2==res_sum:
    print("Result received correctly!")
    print("Expected   = ", hex(res))
    print("Received   = ", hex(res2))
else:
    print("Result INCORRECT!")
    print("Expected   = ", hex(res))
    print("Received   = ", hex(res2))
    print("Offsert:   = ", hex(res2-res))


Result received correctly!
Expected   =  0x19bac79653e722eeaf30958a840a36dbb71b3c46d967f075ba2871383a710414cf5fea190cf42a403eb49d274fd4879aa7f415845a4ac9ddd72e94a919e50213e
Received   =  0x19bac79653e722eeaf30958a840a36dbb71b3c46d967f075ba2871383a710414cf5fea190cf42a403eb49d274fd4879aa7f415845a4ac9ddd72e94a919e50213e


In [31]:
ser.close()